<a href="https://colab.research.google.com/github/QaziSaim/Post-Discharge-Medical-AI-Assistant-POC-/blob/main/Medical_MultiAgent_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install langchain-google-genai
!pip install pypdf
!pip install langchain-community
!pip install langchain_core
!pip install pinecone

In [2]:
from google.colab import userdata
import os
os.environ['GOOGLE_API_KEY'] = userdata.get('GEMINI_API')
os.environ['PINECONE_API_KEY']  = userdata.get('PINECONE_API')
os.environ['HUGGINGFACE_API_KEY'] = userdata.get('HUGGINGFACE_API')

In [8]:
pdf_path = '/content/comprehensive-clinical-nephrology.pdf'

In [4]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
from pinecone import Pinecone
pc = Pinecone(api_key = userdata.get('PINECONE_API'))
index = pc.Index('masnavi-index')

In [10]:
loader = PyPDFLoader(pdf_path)

In [11]:
docs = loader.load()

In [12]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=200)
chunks = splitter.split_documents(docs)


In [ ]:
vectors = []
for i, doc in enumerate(chunks):
  text = doc.page_content.strip()
  emb_vec = model.encode(text).tolist()
  meta = {'page':getattr(doc, 'page_number',None),
          'source':'nephrology_reference_book',
          'content':text[:300]
  }
  vectors.append((f'doc_{i}',emb_vec,meta))


In [ ]:
for i in range(0,len(vectors),100):
  batch = vectors[i:i+100]
  index.upsert(batch)